In [ ]:
import pandas as pd

# Extração da Base de Dados

In [ ]:
DF = pd.read_excel('AdventureWorks.xlsx')

In [ ]:
# Verificar tipos de dados em cada coluna
DF.dtypes

In [ ]:
# Verificar os valores das colunas

DF.isna().sum()

In [ ]:
DF.describe()

# Validação dos Dados

In [ ]:
import pandera as pa

In [ ]:
schema = pa.DataFrameSchema({
    'Data Venda':pa.Column(pa.PandasDtype.DateTime),
    'Data Envio':pa.Column(pa.PandasDtype.DateTime),
    'ID Loja':pa.Column(int),
    'ID Produto':pa.Column(int),
    'ID Cliente':pa.Column(int),
    'No. Venda':pa.Column(pa.PandasDtype.String,),
    'Custo Unitário':pa.Column(float),
    'Preço Unitário':pa.Column(float),
    'Quantidade':pa.Column(int),
    'Valor Desconto':pa.Column(float),
    'Valor Venda':pa.Column(float),
    'Produto':pa.Column(pa.PandasDtype.String),
    'Fabricante':pa.Column(pa.PandasDtype.String),
    'Marca':pa.Column(pa.PandasDtype.String),
    'Classe':pa.Column(pa.PandasDtype.String),
    'Cor':pa.Column(pa.PandasDtype.String),
})


In [ ]:
schema.validate(DF)

In [128]:
# Qual o custo por venda?
DF['Custo Venda'] = DF['Custo Unitário'].mul(DF['Quantidade'])

In [ ]:
# Qual o faturamento por venda? 
DF['Faturamento Venda'] = DF['Valor Venda'] - DF['Custo Venda']

In [ ]:
DF

In [ ]:
len(DF['ID Loja'].unique())

In [ ]:
# Quantas lojas estão presentes no dataset?
print(f'Existem {len(DF["ID Loja"].unique())} lojas. Os ids são: {DF["ID Loja"].unique()}')


In [ ]:
# Quais produtos estão presentes no dataset?
print(f'Existem {len(DF["Produto"].unique())} Produtos. Os produtos são: {(DF["Produto"].unique())}')

In [ ]:
# Criando a coluna de Diferença entre a data da venda e data do envio.
DF['Delta_Dias'] = (DF['Data Envio']-DF['Data Venda']).dt.days

In [ ]:
# Qual das lojas tem a maior média entre a data da venda e data de envio?
DF.groupby(['ID Loja'])['Delta_Dias'].mean().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [15, 6]
plt.rcParams["figure.autolayout"] = True
import numpy as np
import seaborn as sns
sns.set()
medianprops = dict( linewidth=2.5,color='orange')
i=0
for loja in DF['ID Loja'].unique():

    filtro = DF['ID Loja'] == loja
    plt.boxplot(DF.loc[filtro]['Delta_Dias'],labels=[f'Loja {loja}'],positions=[i],widths=0.25,boxprops=dict(linewidth=3),medianprops=medianprops)
    i+=1
plt.title('Média de dias entre a venda e o envio')
plt.grid(True,'major')
plt.show() 

In [ ]:
# Qual o trimestre em que a venda foi realizada?
DF['Trimestre'] = DF['Data Venda'].dt.quarter

In [ ]:
filtro = DF['Data Venda'].dt.year==2009
y=DF.loc[filtro].groupby('Trimestre')['Faturamento Venda'].sum()
plt.bar(x=y.index+4,height=y,label='2009')
filtro = DF['Data Venda'].dt.year==2008
y=DF.loc[filtro].groupby('Trimestre')['Faturamento Venda'].sum()
plt.bar(x=y.index,height=y,label='2008')
plt.xticks([1,2,3,4,5,6,7,8],['2008/1','2008/2','2008/3','2008/4','2009/1','2009/2','2009/3','2009/4'])
plt.legend()
plt.title('Total de Faturamento por Trimestre')


In [ ]:
# Quantos Clientes cada loja atendeu por ano?
filtro = DF['Data Venda'].dt.year==2009
y=DF.loc[filtro].groupby('ID Loja')['ID Cliente'].count()

fig, axs = plt.subplots(2,sharex=True)
fig.suptitle('Evolução de clientes por Trimestre em cada loja')
filtro = DF['Data Venda'].dt.year==2008
y=DF.loc[filtro].groupby(['Trimestre','ID Loja'])['ID Cliente'].count().reset_index()
for i in y['ID Loja'].unique():
    filtro = y['ID Loja'] == i
    axs[0].plot(y.loc[filtro]['Trimestre'],y.loc[filtro]['ID Cliente'],label=f'Loja {i}')
filtro = DF['Data Venda'].dt.year==2009
y=DF.loc[filtro].groupby(['Trimestre','ID Loja'])['ID Cliente'].count().reset_index()
for i in y['ID Loja'].unique():
    filtro = y['ID Loja'] == i
    axs[1].plot(y.loc[filtro]['Trimestre'],y.loc[filtro]['ID Cliente'],label=f'Loja {i}')    
plt.xticks([1,2,3,4])
axs[0].set_title('Ano 2008')
axs[1].set_title('Ano 2009')
plt.legend()